In [3]:
!pip install xgboost

ERROR: Could not find a version that satisfies the requirement igboost (from versions: none)
ERROR: No matching distribution found for igboost


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV

import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('used cars.csv')
df.head()

,Unnamed: 0,id,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,county,state,lat,long
0,55483,7315914053,0,2018.0,ram,promaster 2500,excellent,NaN,gas,44244.0,clean,automatic,NaN,NaN,van,NaN,NaN,ca,32.792800,-116.966500
1,162368,7310885048,13995,2017.0,mazda,cx-3,NaN,4 cylinders,gas,7037.0,rebuilt,automatic,NaN,NaN,SUV,white,NaN,ia,41.207382,-96.023096
2,234393,7308243856,19990,2019.0,mitsubishi,eclipse cross sp,good,NaN,gas,35313.0,clean,other,4wd,NaN,hatchback,white,NaN,nc,35.190000,-80.830000
3,276110,7315817729,0,2019.0,honda,cr-v,NaN,NaN,gas,25626.0,clean,automatic,NaN,NaN,SUV,orange,NaN,ny,40.854573,-74.120219
4,349033,7301620999,42900,2015.0,chevrolet,corvette,excellent,8 cylinders,gas,29000.0,clean,automatic,NaN,NaN,convertible,black,NaN,sc,34.755562,-82.906419


In [6]:
df.shape

(64032, 20)

In [8]:
df.isnull().sum()

Unnamed: 0          0
id                  0
price               0
year              158
manufacturer     2569
model             802
condition       26097
cylinders       26511
fuel              424
odometer          669
title_status     1222
transmission      353
drive           19471
size            45841
type            13785
paint_color     19505
county          64032
state               0
lat              1003
long             1003
dtype: int64

In [7]:
df.describe()

,Unnamed: 0,id,price,year,odometer,county,lat,long
count,64032.000000,6.403200e+04,6.403200e+04,63874.000000,6.336300e+04,0.0,63029.000000,63029.000000
mean,213648.728886,7.311486e+09,2.005202e+04,2011.243542,9.873296e+04,NaN,38.517453,-94.812321
std,123069.399332,4.470434e+06,4.651339e+05,9.442033,2.337273e+05,NaN,5.842070,18.491636
min,15.000000,7.212513e+09,0.000000e+00,1903.000000,0.000000e+00,NaN,-56.073733,-159.719900
25%,106882.750000,7.308130e+09,5.900000e+03,2008.000000,3.771300e+04,NaN,34.667900,-112.010365
50%,214403.500000,7.312665e+09,1.390000e+04,2014.000000,8.500000e+04,NaN,39.171991,-88.527910
75%,319853.500000,7.315256e+09,2.600000e+04,2017.000000,1.330000e+05,NaN,42.427110,-80.819100
max,426872.000000,7.317095e+09,1.134568e+08,2022.000000,1.000000e+07,NaN,64.878089,94.163200


In [9]:
drop_columns = ['Unnamed: 0', 'id', 'title_status', 'size', 'lat', 'long', 'county']
df = df.drop(columns = drop_columns, axis = 1)

In [10]:
df.shape

(64032, 13)

In [11]:
df = df.dropna()

### Filter Categorical features

In [12]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = []

features = df.columns.values.tolist()

for col in features:
    if df[col].dtype in numerics:
        continue
    categorical_columns.append(col)

### Encoding categorical columns using get_dummies

In [14]:
df_dummies = pd.get_dummies(df[categorical_columns], drop_first = False)

In [15]:
df_dummies.head()

,manufacturer_acura,manufacturer_alfa-romeo,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,manufacturer_chevrolet,manufacturer_chrysler,manufacturer_datsun,manufacturer_dodge,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
5,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_dummies.shape

(17491, 4327)

In [18]:
df = df.join(df_dummies)

In [19]:
df.shape

(17491, 4340)

In [20]:
df.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
5,0,2006.0,chrysler,300,like new,8 cylinders,gas,149000.0,automatic,rwd,...,0,0,0,0,0,0,0,0,0,0
9,20995,2011.0,chevrolet,silverado 1500,excellent,8 cylinders,gas,92001.0,automatic,4wd,...,0,0,0,0,0,0,0,1,0,0
15,50995,2017.0,gmc,yukon denali,like new,8 cylinders,gas,70227.0,automatic,4wd,...,0,0,0,0,0,0,0,0,0,0
22,13500,2014.0,chevrolet,tahoe,good,8 cylinders,gas,96007.0,automatic,rwd,...,0,0,0,0,0,0,0,0,0,0
29,34990,2016.0,gmc,canyon crew cab sle pickup,good,6 cylinders,gas,34425.0,other,4wd,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df.drop(columns = categorical_columns, axis = 1, inplace = True)

In [22]:
df.head()

,price,year,odometer,manufacturer_acura,manufacturer_alfa-romeo,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,manufacturer_chevrolet,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
5,0,2006.0,149000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,20995,2011.0,92001.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
15,50995,2017.0,70227.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,13500,2014.0,96007.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
29,34990,2016.0,34425.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Selecting realistic data

In [23]:
df = df[df['price'] > 1000]
df = df[df['price'] < 40000]

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15672 entries, 9 to 64031
Columns: 4330 entries, price to state_wy
dtypes: float64(2), int64(1), uint8(4327)
memory usage: 65.1 MB


In [25]:
X = df.drop(['price'], axis = 1)
y = df['price']

In [27]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 10)

In [28]:
xgb = xgb.XGBRegressor()

In [29]:
xgb.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [30]:
y_pred = xgb.predict(x_test)

In [31]:
r2_score(y_test, y_pred)

0.8710020308271704